# Finance — Overview

## Purpose
Finance is the discipline of managing money, investments, and risk. It provides the mathematical and conceptual frameworks for:
- **Valuing assets** — determining what something is worth today based on future cash flows
- **Managing risk** — quantifying and hedging exposure to adverse outcomes
- **Optimizing portfolios** — balancing return and risk across multiple investments
- **Pricing derivatives** — valuing options, futures, and other complex instruments

## Key Questions This Section Answers
1. How much is a future cash flow worth today? (Time Value of Money)
2. How do we measure and manage financial risk? (VaR, CVaR, Greeks)
3. How do we build optimal portfolios? (Mean-Variance Optimization, Efficient Frontier)
4. How do we price options and derivatives? (Black-Scholes, Binomial Models)
5. How do bonds and fixed income instruments work? (Yield curves, Duration, Convexity)

---

## 1. Time Value of Money (TVM)

The foundational principle: **a dollar today is worth more than a dollar tomorrow**.

### Present Value (PV)
$$PV = \frac{FV}{(1 + r)^n}$$

Where:
- $FV$ = Future Value
- $r$ = discount rate (per period)
- $n$ = number of periods

### Future Value (FV)
$$FV = PV \times (1 + r)^n$$

### Net Present Value (NPV)
$$NPV = \sum_{t=0}^{n} \frac{CF_t}{(1 + r)^t}$$

NPV is used to evaluate investments: **positive NPV = value-creating project**.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Time Value of Money visualization
years = np.arange(0, 31)
rates = [0.02, 0.05, 0.08, 0.10]
pv = 1000  # Initial investment

fig = go.Figure()
for r in rates:
    fv = pv * (1 + r) ** years
    fig.add_trace(go.Scatter(
        x=years, y=fv,
        mode='lines',
        name=f'{r*100:.0f}% rate',
        line=dict(width=2)
    ))

fig.update_layout(
    title='Compound Growth: $1,000 Investment Over 30 Years',
    xaxis_title='Years',
    yaxis_title='Future Value ($)',
    template='plotly_white',
    hovermode='x unified'
)
fig

## 2. Risk and Return

### Expected Return
$$E[R] = \sum_{i=1}^{n} p_i \cdot R_i$$

### Volatility (Standard Deviation)
$$\sigma = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (R_i - \bar{R})^2}$$

### Sharpe Ratio
Measures risk-adjusted return:
$$\text{Sharpe} = \frac{E[R_p] - R_f}{\sigma_p}$$

Where $R_f$ is the risk-free rate.

### Value at Risk (VaR)
The maximum expected loss at a given confidence level:
$$VaR_{\alpha} = -\mu + \sigma \cdot z_{\alpha}$$

For example, 95% VaR means: "We are 95% confident losses won't exceed this amount."

In [ ]:
# Simulate stock returns and visualize risk
np.random.seed(42)
n_days = 252 * 5  # 5 years of daily returns

# Simulate two assets: high-risk/high-return vs low-risk/low-return
returns_risky = np.random.normal(0.0008, 0.02, n_days)  # ~20% annual vol
returns_safe = np.random.normal(0.0003, 0.008, n_days)  # ~8% annual vol

# Cumulative wealth
wealth_risky = 100 * np.cumprod(1 + returns_risky)
wealth_safe = 100 * np.cumprod(1 + returns_safe)

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Portfolio Value Over Time', 'Return Distribution'
))

days = np.arange(n_days)
fig.add_trace(go.Scatter(x=days, y=wealth_risky, name='High Risk', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=days, y=wealth_safe, name='Low Risk', line=dict(color='blue')), row=1, col=1)

fig.add_trace(go.Histogram(x=returns_risky, name='High Risk', opacity=0.6, marker_color='red', nbinsx=50), row=1, col=2)
fig.add_trace(go.Histogram(x=returns_safe, name='Low Risk', opacity=0.6, marker_color='blue', nbinsx=50), row=1, col=2)

# Add VaR lines
var_95_risky = np.percentile(returns_risky, 5)
var_95_safe = np.percentile(returns_safe, 5)
fig.add_vline(x=var_95_risky, line_dash='dash', line_color='red', row=1, col=2)
fig.add_vline(x=var_95_safe, line_dash='dash', line_color='blue', row=1, col=2)

fig.update_layout(
    title='Risk vs Return: High-Risk vs Low-Risk Portfolio',
    template='plotly_white',
    showlegend=True,
    barmode='overlay'
)
fig.update_xaxes(title_text='Days', row=1, col=1)
fig.update_xaxes(title_text='Daily Returns', row=1, col=2)
fig.update_yaxes(title_text='Portfolio Value ($)', row=1, col=1)
fig.update_yaxes(title_text='Frequency', row=1, col=2)
fig.show()

print(f"High Risk - Annual Return: {((wealth_risky[-1]/100)**(252/n_days)-1)*100:.1f}%, Annual Vol: {np.std(returns_risky)*np.sqrt(252)*100:.1f}%")
print(f"Low Risk  - Annual Return: {((wealth_safe[-1]/100)**(252/n_days)-1)*100:.1f}%, Annual Vol: {np.std(returns_safe)*np.sqrt(252)*100:.1f}%")
print(f"\n95% VaR (dashed lines): High Risk = {var_95_risky*100:.2f}%, Low Risk = {var_95_safe*100:.2f}%")

## 3. Portfolio Theory

### Diversification
"Don't put all your eggs in one basket." Combining uncorrelated assets reduces overall portfolio risk.

### Portfolio Variance (2 assets)
$$\sigma_p^2 = w_1^2 \sigma_1^2 + w_2^2 \sigma_2^2 + 2 w_1 w_2 \sigma_1 \sigma_2 \rho_{1,2}$$

Where:
- $w_i$ = weight of asset $i$
- $\sigma_i$ = volatility of asset $i$
- $\rho_{1,2}$ = correlation between assets

### Efficient Frontier
The set of portfolios that offer the highest expected return for a given level of risk (or lowest risk for a given return).

In [ ]:
# Efficient Frontier visualization
np.random.seed(123)

# Asset parameters
returns_mean = np.array([0.12, 0.18, 0.08, 0.15])  # Expected annual returns
returns_std = np.array([0.15, 0.30, 0.10, 0.22])   # Annual volatilities
n_assets = len(returns_mean)

# Correlation matrix
corr = np.array([
    [1.0, 0.3, 0.1, 0.5],
    [0.3, 1.0, 0.2, 0.4],
    [0.1, 0.2, 1.0, 0.3],
    [0.5, 0.4, 0.3, 1.0]
])

# Covariance matrix
cov = np.outer(returns_std, returns_std) * corr

# Generate random portfolios
n_portfolios = 5000
results = np.zeros((n_portfolios, 3))

for i in range(n_portfolios):
    weights = np.random.random(n_assets)
    weights /= weights.sum()
    
    port_return = np.dot(weights, returns_mean)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    sharpe = (port_return - 0.02) / port_vol  # Assuming 2% risk-free rate
    
    results[i] = [port_vol, port_return, sharpe]

# Find optimal portfolios
max_sharpe_idx = results[:, 2].argmax()
min_vol_idx = results[:, 0].argmin()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results[:, 0], y=results[:, 1],
    mode='markers',
    marker=dict(size=4, color=results[:, 2], colorscale='Viridis', 
                colorbar=dict(title='Sharpe Ratio'), showscale=True),
    name='Portfolios',
    text=[f'Sharpe: {s:.2f}' for s in results[:, 2]],
    hovertemplate='Risk: %{x:.1%}<br>Return: %{y:.1%}<br>%{text}'
))

fig.add_trace(go.Scatter(
    x=[results[max_sharpe_idx, 0]], y=[results[max_sharpe_idx, 1]],
    mode='markers', marker=dict(size=15, color='red', symbol='star'),
    name='Max Sharpe'
))

fig.add_trace(go.Scatter(
    x=[results[min_vol_idx, 0]], y=[results[min_vol_idx, 1]],
    mode='markers', marker=dict(size=15, color='green', symbol='diamond'),
    name='Min Volatility'
))

fig.update_layout(
    title='Efficient Frontier: Portfolio Optimization',
    xaxis_title='Annual Volatility (Risk)',
    yaxis_title='Expected Annual Return',
    template='plotly_white',
    xaxis=dict(tickformat='.0%'),
    yaxis=dict(tickformat='.0%')
)
fig

## 4. Derivatives and Options

### Options Basics
- **Call Option**: Right to BUY at strike price
- **Put Option**: Right to SELL at strike price

### Black-Scholes Formula (European Call)
$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

### The Greeks
| Greek | Measures | Formula |
|-------|----------|--------|
| Delta ($\Delta$) | Price sensitivity | $\frac{\partial V}{\partial S}$ |
| Gamma ($\Gamma$) | Delta sensitivity | $\frac{\partial^2 V}{\partial S^2}$ |
| Theta ($\Theta$) | Time decay | $\frac{\partial V}{\partial t}$ |
| Vega ($\nu$) | Volatility sensitivity | $\frac{\partial V}{\partial \sigma}$ |
| Rho ($\rho$) | Interest rate sensitivity | $\frac{\partial V}{\partial r}$ |

In [ ]:
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """Black-Scholes call option price."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def delta(S, K, T, r, sigma):
    """Delta of a call option."""
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

# Parameters
K = 100  # Strike price
T = 1.0  # Time to expiry (1 year)
r = 0.05  # Risk-free rate
sigma = 0.20  # Volatility

# Range of spot prices
S_range = np.linspace(60, 140, 100)

# Calculate option prices and payoffs
call_prices = [black_scholes_call(S, K, T, r, sigma) for S in S_range]
payoff = np.maximum(S_range - K, 0)
deltas = [delta(S, K, T, r, sigma) for S in S_range]

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Call Option Value vs Spot Price', 'Delta (Hedge Ratio)'
))

fig.add_trace(go.Scatter(x=S_range, y=call_prices, name='Option Value', line=dict(color='blue', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=S_range, y=payoff, name='Intrinsic Value', line=dict(color='red', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=S_range, y=deltas, name='Delta', line=dict(color='green', width=2)), row=1, col=2)

fig.add_vline(x=K, line_dash='dot', line_color='gray', row=1, col=1)
fig.add_vline(x=K, line_dash='dot', line_color='gray', row=1, col=2)
fig.add_hline(y=0.5, line_dash='dot', line_color='gray', row=1, col=2)

fig.update_layout(
    title=f'Black-Scholes Call Option (K={K}, σ={sigma*100}%, T={T}yr)',
    template='plotly_white'
)
fig.update_xaxes(title_text='Spot Price ($)', row=1, col=1)
fig.update_xaxes(title_text='Spot Price ($)', row=1, col=2)
fig.update_yaxes(title_text='Option Value ($)', row=1, col=1)
fig.update_yaxes(title_text='Delta', row=1, col=2)
fig

## 5. Fixed Income

### Bond Pricing
$$P = \sum_{t=1}^{n} \frac{C}{(1+y)^t} + \frac{F}{(1+y)^n}$$

Where:
- $C$ = coupon payment
- $F$ = face value
- $y$ = yield to maturity
- $n$ = number of periods

### Duration
Measures bond price sensitivity to interest rate changes:
$$D = \frac{1}{P} \sum_{t=1}^{n} t \cdot \frac{CF_t}{(1+y)^t}$$

**Rule of thumb**: If rates rise by 1%, a bond with duration of 5 years loses ~5% in value.

In [ ]:
# Yield curve and bond pricing
def bond_price(face, coupon_rate, ytm, years):
    """Calculate bond price."""
    coupon = face * coupon_rate
    periods = np.arange(1, years + 1)
    pv_coupons = sum(coupon / (1 + ytm)**t for t in periods)
    pv_face = face / (1 + ytm)**years
    return pv_coupons + pv_face

# Yield curve scenarios
maturities = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])
normal_curve = np.array([2.0, 2.2, 2.5, 2.8, 3.0, 3.3, 3.5, 3.7, 4.0, 4.1]) / 100
inverted_curve = np.array([4.5, 4.3, 4.0, 3.7, 3.5, 3.2, 3.0, 2.8, 2.5, 2.4]) / 100
flat_curve = np.array([3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]) / 100

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Yield Curve Shapes', 'Bond Price vs Yield'
))

fig.add_trace(go.Scatter(x=maturities, y=normal_curve*100, name='Normal (Upward)', mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=maturities, y=inverted_curve*100, name='Inverted (Recession Signal)', mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=maturities, y=flat_curve*100, name='Flat', mode='lines+markers'), row=1, col=1)

# Bond price sensitivity
yields = np.linspace(0.01, 0.10, 50)
price_5yr = [bond_price(100, 0.04, y, 5) for y in yields]
price_10yr = [bond_price(100, 0.04, y, 10) for y in yields]
price_30yr = [bond_price(100, 0.04, y, 30) for y in yields]

fig.add_trace(go.Scatter(x=yields*100, y=price_5yr, name='5-Year Bond'), row=1, col=2)
fig.add_trace(go.Scatter(x=yields*100, y=price_10yr, name='10-Year Bond'), row=1, col=2)
fig.add_trace(go.Scatter(x=yields*100, y=price_30yr, name='30-Year Bond'), row=1, col=2)

fig.update_layout(title='Fixed Income: Yield Curves and Bond Pricing', template='plotly_white')
fig.update_xaxes(title_text='Maturity (Years)', row=1, col=1)
fig.update_xaxes(title_text='Yield to Maturity (%)', row=1, col=2)
fig.update_yaxes(title_text='Yield (%)', row=1, col=1)
fig.update_yaxes(title_text='Bond Price ($)', row=1, col=2)
fig

## Core Topics in This Section

| Topic | Description |
|-------|-------------|
| **Time Value of Money** | Present/future value, NPV, IRR, annuities |
| **Derivatives** | Options, futures, swaps, Black-Scholes, Greeks |
| **Risk** | VaR, CVaR, volatility, risk-adjusted returns |
| **Portfolio Theory** | Diversification, CAPM, Efficient Frontier, Fama-French |
| **Fixed Income** | Bonds, yield curves, duration, convexity |
| **Econometrics** | Time series analysis, regression, GARCH models |

---

## Key Takeaways

1. **Time value of money** is the foundation — always discount future cash flows
2. **Risk and return are linked** — higher returns require accepting more risk
3. **Diversification works** — combining assets reduces portfolio risk
4. **Options provide asymmetric payoffs** — limited downside, unlimited upside
5. **Duration measures interest rate risk** — longer bonds are more sensitive

## References
- Hull, J. (2018). *Options, Futures, and Other Derivatives*
- Bodie, Kane, Marcus. *Investments*
- Fabozzi, F. *Fixed Income Analysis*